In [2]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import os
from google.colab import drive

# Montar Google Drive para utilizar archivos (Los subi a mi drive)
drive.mount("/content/drive/")

Mounted at /content/drive/


In [3]:
#Ruta de la carpeta que contiene los archivos CSV
ruta_de_acceso = "/content/drive/MyDrive/Dataset de ventas2-20230916T235839Z-001/Dataset de ventas2"

archivos = os.listdir(ruta_de_acceso)
archivos

['Dataset_de_ventas_Abril.csv',
 'Dataset_de_ventas_Agosto.csv',
 'Dataset_de_ventas_Diciembre.csv',
 'Dataset_de_ventas_Enero.csv',
 'Dataset_de_ventas_Febrero.csv',
 'Dataset_de_ventas_Julio.csv',
 'Dataset_de_ventas_Junio.csv',
 'Dataset_de_ventas_Marzo.csv',
 'Dataset_de_ventas_Mayo.csv',
 'Dataset_de_ventas_Noviembre.csv',
 'Dataset_de_ventas_Octubre.csv',
 'Dataset_de_ventas_Septiembre.csv']

In [4]:
dataframes = {}

for archivo in archivos:
  if archivo.endswith('.csv'): # Verificamos los archivos 'CSV'
    mes = archivo.split('_')[3].split('.')[0] # Obtenemos el nombre del 'MES' sin la extension
    ruta_completa = os.path.join(ruta_de_acceso, archivo)  # Ruta completa al archivo
    df = pd.read_csv(ruta_completa)  # Cargar el archivo CSV en un DataFrame
    df_ventas = df.copy() #Realizamos una copia de df_ventas

    df_ventas['Cantidad Pedida'] = pd.to_numeric(df_ventas['Cantidad Pedida'], errors='coerce')  #El errors= 'coerce' transforma los valores no numericos a NaN'
    df_ventas['Precio Unitario'] = pd.to_numeric(df_ventas['Precio Unitario'], errors='coerce')


    df_ventas = df_ventas.loc[~df_ventas['Cantidad Pedida'].isna()]
    df_ventas = df_ventas.loc[~df_ventas['Precio Unitario'].isna()]
    df_ventas1 = df_ventas.dropna()

    # Transformo todos los valores numericos al tipo de dato int
    df_ventas1['Cantidad Pedida'] = df_ventas1['Cantidad Pedida'].astype(int)
    df_ventas1['Precio Unitario'] = df_ventas1['Precio Unitario'].astype(int)

    dataframes[mes] = df_ventas1  # Agregar el DataFrame al diccionario usando el nombre del mes

In [43]:
pd.set_option('display.float_format', '{:.2f}'.format)

Analisis_de_productos = dataframes['Enero'].groupby('Producto').agg({
    'Cantidad Pedida' : 'sum',
    'Precio Unitario' : 'mean'
})
# Calculo los Productos Pedidos por Estado en el Mes de Enero
direccion_df = dataframes['Enero']['Dirección de Envio'].str.split(', ', expand = True)
direccion_df.columns = ['Calle','Ciudad','Cod Post Estado']
direccion_df[['Estado', 'Cod Post']] = direccion_df['Cod Post Estado'].str.split(' ', expand = True)
df_ventas_desglosado = pd.concat([dataframes['Enero'], direccion_df], axis = True)
ventas_por_estado_mesEnero = df_ventas_desglosado.groupby('Estado')['Cantidad Pedida'].sum()
ventas_por_estado_mesEnero = ventas_por_estado_mesEnero.reset_index(inplace = False)

# Calculo las Unidades Vendidas y Precio de los Productos del Mes de Enero
Analisis_de_productos1 = Analisis_de_productos.sort_values(by = 'Cantidad Pedida', ascending = False)
Analisis_de_productos1 = Analisis_de_productos1.reset_index(inplace = False)
Analisis_de_productos1
Productos = Analisis_de_productos1['Producto']
Unidades = Analisis_de_productos1['Cantidad Pedida']
Precio = Analisis_de_productos1['Precio Unitario']

# Calculo las Unidades Vendidas y por Hora del Mes de Enero
df1_ventas = dataframes['Enero']
df1_ventas['Fecha de Compra'] = pd.to_datetime(df1_ventas['Fecha de Pedido'])
df1_ventas
def extraer_datos(datos):
  datos['Hora'] = datos['Fecha de Compra'].dt.hour
  return datos
df2_ventas = extraer_datos(df1_ventas)
df2_ventas
horas = list(range(24))
ventas_por_hora_mesEnero = df2_ventas.groupby('Hora')['Cantidad Pedida'].sum()

# Calculo las Unidades Vendidas por Dia del Mes de Enero
def extraer_datos(datos):
  datos['Dia'] = datos['Fecha de Compra'].dt.day
  return datos
df3_ventas = extraer_datos(df2_ventas)
df3_ventas
ventas_por_dia = df3_ventas.groupby('Dia')['Cantidad Pedida'].sum()
dias = list(ventas_por_dia.index)

# Calculo el Valor Total de los Productos Pedidos por Dia del Mes de Enero
valor_total_por_dia_Enero = df1_ventas.groupby('Dia')['Cantidad Pedida', 'Precio Unitario'].sum()
valor_total_por_dia_Enero['Valor Total'] = valor_total_por_dia_Enero['Cantidad Pedida'] * valor_total_por_dia_Enero['Precio Unitario']
dias1 = list(valor_total_por_dia_Enero.index)
df4_ventas = valor_total_por_dia_Enero

# Gráfico de las Ventas por Estado del Mes de Enero
fig = px.choropleth(ventas_por_estado_mesEnero, locations = 'Estado', locationmode = 'USA-states',
                    color = 'Cantidad Pedida', color_continuous_scale = 'RdYlGn', scope = 'usa',
                    title = 'ventas por Estado'
                    )
fig.show()

# Gráfico de las Unidades Vendidas y Precio de los Productos del Mes de Enero
def grafico_unidades_precio(productos, unidades, precio):
    fig = go.Figure()
    fig.add_trace(go.Bar(
        x=productos,
        y=unidades,
        name='Unidades Vendidas',
        marker=dict(color=unidades)
    ))
    fig.add_trace(go.Scatter(
        x=productos,
        y=precio,
        name='Precio Unitario',
        line=dict(color='Red', shape='spline', smoothing=1.3, width=3),
        yaxis='y2'
    ))
    fig.update_layout(
        title='Unidades Vendidas y Precio de Productos Mes de Enero',
        xaxis=dict(title='Productos'),
        yaxis=dict(title='Unidades', side='left'),
        yaxis2=dict(title='Precio', overlaying='y', side='right'),
        yaxis_range=[0, max(unidades) * 1.1],
        yaxis2_range=[0, max(precio) * 1.1 ]
    )
    fig.show()

# Gráfico de las Unidades Vendidas por Hora del Mes de Enero
def grafico_unidades_por_hora(horas, ventas_por_hora_mesEnero):
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=horas,
        y=ventas_por_hora_mesEnero,
        name='Ventas por Hora',
        line=dict(color='Red', shape='spline', smoothing=0.1, width=2)
    ))
    fig.update_layout(
        title='Unidades Vendidas por Hora Mes de Enero',
        xaxis=dict(title='Horas', tickvals=horas),
        yaxis=dict(title='Unidades Vendidas', side='left')
    )
    fig.show()

# Gráfico de las Unidades Vendidas por Dia del Mes de Enero
def grafico_unidades_por_dia(dias, ventas_por_dia):
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=dias,
        y=ventas_por_dia,
        name='Ventas por Dia',
        line=dict(color='Red', shape='spline', smoothing=0.5, width=2)
    ))
    fig.update_layout(
        title='Unidades Vendidas por Dia Mes de Enero',
        xaxis=dict(title='Dias', tickvals=dias),
        yaxis=dict(title='Unidades Vendidas', side='left')
    )
    fig.show()

# Gráfico del Valor Total de los Productos Pedidos por Dia del Mes de Enero
def grafico_valor_total(dias1, valor_total_por_dia_Enero):
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=df4_ventas.index,
        y=df4_ventas['Valor Total'],
        marker=dict(size=6),
        line=dict(color='Red', shape='spline', smoothing=0.5, width=2)
    ))
    fig.update_layout(
        title='Valor Total de los Productos Pedidos por Dia del Mes de Enero ',
        xaxis=dict(title='Dia', dtick=1),
        yaxis=dict(title='Valor Total'),
        showlegend=False
    )
    fig.show()

# LLamo a las funciones para mostrar los gráficos
grafico_unidades_precio(Productos, Unidades, Precio)
grafico_unidades_por_hora(horas, ventas_por_hora_mesEnero)
grafico_unidades_por_dia(dias, ventas_por_dia)
grafico_valor_total(dias1, valor_total_por_dia_Enero)

<ipython-input-43-ac0dba71da45>:45: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



In [44]:
df_valorTotal_Enero = valor_total_por_dia_Enero['Valor Total'].sum()
print("Valor Total del Mes de Enero: $", df_valorTotal_Enero)

Valor Total del Mes de Enero: $ 632580236


In [8]:
pd.set_option('display.float_format', '{:.2f}'.format)

Analisis_de_productos = dataframes['Febrero'].groupby('Producto').agg({
    'Cantidad Pedida' : 'sum',
    'Precio Unitario' : 'mean'
})
# Calculo los Productos Pedidos por Estado en el Mes de Febrero
direccion_df = dataframes['Febrero']['Dirección de Envio'].str.split(', ', expand=True)
direccion_df.columns = ['Calle', 'Ciudad', 'Cod Post Estado']
direccion_df[['Estado', 'Cod Post']] = direccion_df['Cod Post Estado'].str.split(' ', expand=True)
df_ventas_desglosado = pd.concat([dataframes['Febrero'], direccion_df], axis=True)
ventas_por_estado_mesFebrero = df_ventas_desglosado.groupby('Estado')['Cantidad Pedida'].sum()
ventas_por_estado_mesFebrero = ventas_por_estado_mesFebrero.reset_index(inplace=False)

# Calculo las Unidades Vendidas y Precio de los Productos del Mes de Febrero
Analisis_de_productos1 = Analisis_de_productos.sort_values(by='Cantidad Pedida', ascending=False)
Analisis_de_productos1 = Analisis_de_productos1.reset_index(inplace=False)
Analisis_de_productos1
Productos = Analisis_de_productos1['Producto']
Unidades = Analisis_de_productos1['Cantidad Pedida']
Precio = Analisis_de_productos1['Precio Unitario']

# Calculo las Unidades Vendidas y por Hora del Mes de Febrero
df1_ventas = dataframes['Febrero']
df1_ventas['Fecha de Compra'] = pd.to_datetime(df1_ventas['Fecha de Pedido'])
df1_ventas
def extraer_datos(datos):
    datos['Hora'] = datos['Fecha de Compra'].dt.hour
    return datos
df2_ventas = extraer_datos(df1_ventas)
df2_ventas
horas = list(range(24))
ventas_por_hora_mesFebrero = df2_ventas.groupby('Hora')['Cantidad Pedida'].sum()

# Calculo las Unidades Vendidas por Dia del Mes de Febrero
def extraer_datos(datos):
    datos['Dia'] = datos['Fecha de Compra'].dt.day
    return datos
df3_ventas = extraer_datos(df2_ventas)
df3_ventas
ventas_por_dia = df3_ventas.groupby('Dia')['Cantidad Pedida'].sum()
dias = list(ventas_por_dia.index)

# Calculo el Valor Total de los Productos Pedidos por Dia del Mes de Febrero
valor_total_por_dia_Febrero = df1_ventas.groupby('Dia')['Cantidad Pedida', 'Precio Unitario'].sum()
valor_total_por_dia_Febrero['Valor Total'] = valor_total_por_dia_Febrero['Cantidad Pedida'] * valor_total_por_dia_Febrero['Precio Unitario']
dias1 = list(valor_total_por_dia_Febrero.index)
df4_ventas = valor_total_por_dia_Febrero

# Gráfico de las Ventas por Estado del Mes de Febrero
fig = px.choropleth(ventas_por_estado_mesFebrero, locations='Estado', locationmode='USA-states',
                    color='Cantidad Pedida', color_continuous_scale='RdYlGn', scope='usa',
                    title='Ventas por Estado en Febrero'
                    )
fig.show()

# Gráfico de las Unidades Vendidas y Precio de los Productos del Mes de Febrero
def grafico_unidades_precio(productos, unidades, precio):
    fig = go.Figure()
    fig.add_trace(go.Bar(
        x=productos,
        y=unidades,
        name='Unidades Vendidas',
        marker=dict(color=unidades)
    ))
    fig.add_trace(go.Scatter(
        x=productos,
        y=precio,
        name='Precio Unitario',
        line=dict(color='Red', shape='spline', smoothing=1.3, width=3),
        yaxis='y2'
    ))
    fig.update_layout(
        title='Unidades Vendidas y Precio de Productos Mes de Febrero',
        xaxis=dict(title='Productos'),
        yaxis=dict(title='Unidades', side='left'),
        yaxis2=dict(title='Precio', overlaying='y', side='right'),
        yaxis_range=[0, max(unidades) * 1.1],
        yaxis2_range=[0, max(precio) * 1.1 ]
    )
    fig.show()

# Gráfico de las Unidades Vendidas por Hora del Mes de Febrero
def grafico_unidades_por_hora(horas, ventas_por_hora_mesFebrero):
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=horas,
        y=ventas_por_hora_mesFebrero,
        name='Ventas por Hora',
        line=dict(color='Red', shape='spline', smoothing=0.1, width=2)
    ))
    fig.update_layout(
        title='Unidades Vendidas por Hora Mes de Febrero',
        xaxis=dict(title='Horas', tickvals=horas),
        yaxis=dict(title='Unidades Vendidas', side='left')
    )
    fig.show()

# Gráfico de las Unidades Vendidas por Día del Mes de Febrero
def grafico_unidades_por_dia(dias, ventas_por_dia):
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=dias,
        y=ventas_por_dia,
        name='Ventas por Día',
        line=dict(color='Red', shape='spline', smoothing=0.5, width=2)
    ))
    fig.update_layout(
        title='Unidades Vendidas por Día Mes de Febrero',
        xaxis=dict(title='Días', tickvals=dias),
        yaxis=dict(title='Unidades Vendidas', side='left')
    )
    fig.show()

# Gráfico del Valor Total de los Productos Pedidos por Día del Mes de Febrero
def grafico_valor_total(dias1, valor_total_por_dia_Febrero):
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=dias1,
        y=valor_total_por_dia_Febrero['Valor Total'],
        marker=dict(size=6),
        line=dict(color='Red', shape='spline', smoothing=0.5, width=2)
    ))
    fig.update_layout(
        title='Valor Total de los Productos Pedidos por Día del Mes de Febrero',
        xaxis=dict(title='Día', dtick=1),
        yaxis=dict(title='Valor Total'),
        showlegend=False
    )
    fig.show()

# Llamo a las funciones para mostrar los gráficos
grafico_unidades_precio(Productos, Unidades, Precio)
grafico_unidades_por_hora(horas, ventas_por_hora_mesFebrero)
grafico_unidades_por_dia(dias, ventas_por_dia)
grafico_valor_total(dias1, valor_total_por_dia_Febrero)



<ipython-input-8-3d7f935ea594>:45: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  valor_total_por_dia_Febrero = df1_ventas.groupby('Dia')['Cantidad Pedida', 'Precio Unitario'].sum()


In [42]:
df_valorTotal_Febrero = valor_total_por_dia_Febrero['Valor Total'].sum()
print("Valor Total del Mes de Febrero: $", df_valorTotal_Febrero)

Valor Total del Mes de Febrero: $ 1051697796


In [9]:
pd.set_option('display.float_format', '{:.2f}'.format)

Analisis_de_productos = dataframes['Marzo'].groupby('Producto').agg({
    'Cantidad Pedida' : 'sum',
    'Precio Unitario' : 'mean'
})
# Calculo los Productos Pedidos por Estado en el Mes de Marzo
direccion_df = dataframes['Marzo']['Dirección de Envio'].str.split(', ', expand=True)
direccion_df.columns = ['Calle', 'Ciudad', 'Cod Post Estado']
direccion_df[['Estado', 'Cod Post']] = direccion_df['Cod Post Estado'].str.split(' ', expand=True)
df_ventas_desglosado = pd.concat([dataframes['Marzo'], direccion_df], axis=True)
ventas_por_estado_mesMarzo = df_ventas_desglosado.groupby('Estado')['Cantidad Pedida'].sum()
ventas_por_estado_mesMarzo = ventas_por_estado_mesMarzo.reset_index(inplace=False)

# Calculo las Unidades Vendidas y Precio de los Productos del Mes de Marzo
Analisis_de_productos1 = Analisis_de_productos.sort_values(by='Cantidad Pedida', ascending=False)
Analisis_de_productos1 = Analisis_de_productos1.reset_index(inplace=False)
Analisis_de_productos1
Productos = Analisis_de_productos1['Producto']
Unidades = Analisis_de_productos1['Cantidad Pedida']
Precio = Analisis_de_productos1['Precio Unitario']

# Calculo las Unidades Vendidas y por Hora del Mes de Marzo
df1_ventas = dataframes['Marzo']
df1_ventas['Fecha de Compra'] = pd.to_datetime(df1_ventas['Fecha de Pedido'])
df1_ventas
def extraer_datos(datos):
    datos['Hora'] = datos['Fecha de Compra'].dt.hour
    return datos
df2_ventas = extraer_datos(df1_ventas)
df2_ventas
horas = list(range(24))
ventas_por_hora_mesMarzo = df2_ventas.groupby('Hora')['Cantidad Pedida'].sum()

# Calculo las Unidades Vendidas por Dia del Mes de Marzo
def extraer_datos(datos):
    datos['Dia'] = datos['Fecha de Compra'].dt.day
    return datos
df3_ventas = extraer_datos(df2_ventas)
df3_ventas
ventas_por_dia = df3_ventas.groupby('Dia')['Cantidad Pedida'].sum()
dias = list(ventas_por_dia.index)

# Calculo el Valor Total de los Productos Pedidos por Dia del Mes de Marzo
valor_total_por_dia_Marzo = df1_ventas.groupby('Dia')['Cantidad Pedida', 'Precio Unitario'].sum()
valor_total_por_dia_Marzo['Valor Total'] = valor_total_por_dia_Marzo['Cantidad Pedida'] * valor_total_por_dia_Marzo['Precio Unitario']
dias1 = list(valor_total_por_dia_Marzo.index)
df4_ventas = valor_total_por_dia_Marzo

# Gráfico de las Ventas por Estado del Mes de Marzo
fig = px.choropleth(ventas_por_estado_mesMarzo, locations='Estado', locationmode='USA-states',
                    color='Cantidad Pedida', color_continuous_scale='RdYlGn', scope='usa',
                    title='Ventas por Estado en Marzo'
                    )
fig.show()

# Gráfico de las Unidades Vendidas y Precio de los Productos del Mes de Marzo
def grafico_unidades_precio(productos, unidades, precio):
    fig = go.Figure()
    fig.add_trace(go.Bar(
        x=productos,
        y=unidades,
        name='Unidades Vendidas',
        marker=dict(color=unidades)
    ))
    fig.add_trace(go.Scatter(
        x=productos,
        y=precio,
        name='Precio Unitario',
        line=dict(color='Red', shape='spline', smoothing=1.3, width=3),
        yaxis='y2'
    ))
    fig.update_layout(
        title='Unidades Vendidas y Precio de Productos Mes de Marzo',
        xaxis=dict(title='Productos'),
        yaxis=dict(title='Unidades', side='left'),
        yaxis2=dict(title='Precio', overlaying='y', side='right'),
        yaxis_range=[0, max(unidades) * 1.1],
        yaxis2_range=[0, max(precio) * 1.1 ]
    )
    fig.show()

# Gráfico de las Unidades Vendidas por Hora del Mes de Marzo
def grafico_unidades_por_hora(horas, ventas_por_hora_mesMarzo):
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=horas,
        y=ventas_por_hora_mesMarzo,
        name='Ventas por Hora',
        line=dict(color='Red', shape='spline', smoothing=0.1, width=2)
    ))
    fig.update_layout(
        title='Unidades Vendidas por Hora Mes de Marzo',
        xaxis=dict(title='Horas', tickvals=horas),
        yaxis=dict(title='Unidades Vendidas', side='left')
    )
    fig.show()

# Gráfico de las Unidades Vendidas por Día del Mes de Marzo
def grafico_unidades_por_dia(dias, ventas_por_dia):
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=dias,
        y=ventas_por_dia,
        name='Ventas por Día',
        line=dict(color='Red', shape='spline', smoothing=0.5, width=2)
    ))
    fig.update_layout(
        title='Unidades Vendidas por Día Mes de Marzo',
        xaxis=dict(title='Días', tickvals=dias),
        yaxis=dict(title='Unidades Vendidas', side='left')
    )
    fig.show()

# Gráfico del Valor Total de los Productos Pedidos por Día del Mes de Marzo
def grafico_valor_total(dias1, valor_total_por_dia_Marzo):
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=dias1,
        y=valor_total_por_dia_Marzo['Valor Total'],
        marker=dict(size=6),
        line=dict(color='Red', shape='spline', smoothing=0.5, width=2)
    ))
    fig.update_layout(
        title='Valor Total de los Productos Pedidos por Día del Mes de Marzo',
        xaxis=dict(title='Día', dtick=1),
        yaxis=dict(title='Valor Total'),
        showlegend=False
    )
    fig.show()

# Llamo a las funciones para mostrar los gráficos
grafico_unidades_precio(Productos, Unidades, Precio)
grafico_unidades_por_hora(horas, ventas_por_hora_mesMarzo)
grafico_unidades_por_dia(dias, ventas_por_dia)
grafico_valor_total(dias1, valor_total_por_dia_Marzo)


<ipython-input-9-b7bd8d1f514a>:45: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



In [39]:
df_valorTotal_Marzo = valor_total_por_dia_Marzo['Valor Total'].sum()
print("Valor Total del Mes de Marzo: $", df_valorTotal_Marzo)

Valor Total del Mes de Marzo: $ 1526826315


In [10]:
pd.set_option('display.float_format', '{:.2f}'.format)

Analisis_de_productos = dataframes['Abril'].groupby('Producto').agg({
    'Cantidad Pedida' : 'sum',
    'Precio Unitario' : 'mean'
})
# Calculo los Productos Pedidos por Estado en el Mes de Abril
direccion_df = dataframes['Abril']['Dirección de Envio'].str.split(', ', expand=True)
direccion_df.columns = ['Calle', 'Ciudad', 'Cod Post Estado']
direccion_df[['Estado', 'Cod Post']] = direccion_df['Cod Post Estado'].str.split(' ', expand=True)
df_ventas_desglosado = pd.concat([dataframes['Abril'], direccion_df], axis=True)
ventas_por_estado_mesAbril = df_ventas_desglosado.groupby('Estado')['Cantidad Pedida'].sum()
ventas_por_estado_mesAbril = ventas_por_estado_mesAbril.reset_index(inplace=False)

# Calculo las Unidades Vendidas y Precio de los Productos del Mes de Abril
Analisis_de_productos1 = Analisis_de_productos.sort_values(by='Cantidad Pedida', ascending=False)
Analisis_de_productos1 = Analisis_de_productos1.reset_index(inplace=False)
Analisis_de_productos1
Productos = Analisis_de_productos1['Producto']
Unidades = Analisis_de_productos1['Cantidad Pedida']
Precio = Analisis_de_productos1['Precio Unitario']

# Calculo las Unidades Vendidas y por Hora del Mes de Abril
df1_ventas = dataframes['Abril']
df1_ventas['Fecha de Compra'] = pd.to_datetime(df1_ventas['Fecha de Pedido'])
df1_ventas
def extraer_datos(datos):
    datos['Hora'] = datos['Fecha de Compra'].dt.hour
    return datos
df2_ventas = extraer_datos(df1_ventas)
df2_ventas
horas = list(range(24))
ventas_por_hora_mesAbril = df2_ventas.groupby('Hora')['Cantidad Pedida'].sum()

# Calculo las Unidades Vendidas por Dia del Mes de Abril
def extraer_datos(datos):
    datos['Dia'] = datos['Fecha de Compra'].dt.day
    return datos
df3_ventas = extraer_datos(df2_ventas)
df3_ventas
ventas_por_dia = df3_ventas.groupby('Dia')['Cantidad Pedida'].sum()
dias = list(ventas_por_dia.index)

# Calculo el Valor Total de los Productos Pedidos por Dia del Mes de Abril
valor_total_por_dia_Abril = df1_ventas.groupby('Dia')['Cantidad Pedida', 'Precio Unitario'].sum()
valor_total_por_dia_Abril['Valor Total'] = valor_total_por_dia_Abril['Cantidad Pedida'] * valor_total_por_dia_Abril['Precio Unitario']
dias1 = list(valor_total_por_dia_Abril.index)
df4_ventas = valor_total_por_dia_Abril

# Gráfico de las Ventas por Estado del Mes de Abril
fig = px.choropleth(ventas_por_estado_mesAbril, locations='Estado', locationmode='USA-states',
                    color='Cantidad Pedida', color_continuous_scale='RdYlGn', scope='usa',
                    title='Ventas por Estado en Abril'
                    )
fig.show()

# Gráfico de las Unidades Vendidas y Precio de los Productos del Mes de Abril
def grafico_unidades_precio(productos, unidades, precio):
    fig = go.Figure()
    fig.add_trace(go.Bar(
        x=productos,
        y=unidades,
        name='Unidades Vendidas',
        marker=dict(color=unidades)
    ))
    fig.add_trace(go.Scatter(
        x=productos,
        y=precio,
        name='Precio Unitario',
        line=dict(color='Red', shape='spline', smoothing=1.3, width=3),
        yaxis='y2'
    ))
    fig.update_layout(
        title='Unidades Vendidas y Precio de Productos Mes de Abril',
        xaxis=dict(title='Productos'),
        yaxis=dict(title='Unidades', side='left'),
        yaxis2=dict(title='Precio', overlaying='y', side='right'),
        yaxis_range=[0, max(unidades) * 1.1],
        yaxis2_range=[0, max(precio) * 1.1 ]
    )
    fig.show()

# Gráfico de las Unidades Vendidas por Hora del Mes de Abril
def grafico_unidades_por_hora(horas, ventas_por_hora_mesAbril):
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=horas,
        y=ventas_por_hora_mesAbril,
        name='Ventas por Hora',
        line=dict(color='Red', shape='spline', smoothing=0.1, width=2)
    ))
    fig.update_layout(
        title='Unidades Vendidas por Hora Mes de Abril',
        xaxis=dict(title='Horas', tickvals=horas),
        yaxis=dict(title='Unidades Vendidas', side='left')
    )
    fig.show()

# Gráfico de las Unidades Vendidas por Día del Mes de Abril
def grafico_unidades_por_dia(dias, ventas_por_dia):
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=dias,
        y=ventas_por_dia,
        name='Ventas por Día',
        line=dict(color='Red', shape='spline', smoothing=0.5, width=2)
    ))
    fig.update_layout(
        title='Unidades Vendidas por Día Mes de Abril',
        xaxis=dict(title='Días', tickvals=dias),
        yaxis=dict(title='Unidades Vendidas', side='left')
    )
    fig.show()

# Gráfico del Valor Total de los Productos Pedidos por Día del Mes de Abril
def grafico_valor_total(dias1, valor_total_por_dia_Abril):
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=dias1,
        y=valor_total_por_dia_Abril['Valor Total'],
        marker=dict(size=6),
        line=dict(color='Red', shape='spline', smoothing=0.5, width=2)
    ))
    fig.update_layout(
        title='Valor Total de los Productos Pedidos por Día del Mes de Abril',
        xaxis=dict(title='Día', dtick=1),
        yaxis=dict(title='Valor Total'),
        showlegend=False
    )
    fig.show()

# Llamo a las funciones para mostrar los gráficos
grafico_unidades_precio(Productos, Unidades, Precio)
grafico_unidades_por_hora(horas, ventas_por_hora_mesAbril)
grafico_unidades_por_dia(dias, ventas_por_dia)
grafico_valor_total(dias1, valor_total_por_dia_Abril)


<ipython-input-10-2f0f472931cf>:45: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



In [38]:
df_valorTotal_Abril = valor_total_por_dia_Abril['Valor Total'].sum()
print("Valor Total del Mes de Abril: $", df_valorTotal_Abril)

Valor Total del Mes de Abril: $ 2308112142


In [11]:
pd.set_option('display.float_format', '{:.2f}'.format)

Analisis_de_productos = dataframes['Mayo'].groupby('Producto').agg({
    'Cantidad Pedida' : 'sum',
    'Precio Unitario' : 'mean'
})
# Calculo los Productos Pedidos por Estado en el Mes de Mayo
direccion_df = dataframes['Mayo']['Dirección de Envio'].str.split(', ', expand=True)
direccion_df.columns = ['Calle', 'Ciudad', 'Cod Post Estado']
direccion_df[['Estado', 'Cod Post']] = direccion_df['Cod Post Estado'].str.split(' ', expand=True)
df_ventas_desglosado = pd.concat([dataframes['Mayo'], direccion_df], axis=True)
ventas_por_estado_mesMayo = df_ventas_desglosado.groupby('Estado')['Cantidad Pedida'].sum()
ventas_por_estado_mesMayo = ventas_por_estado_mesMayo.reset_index(inplace=False)

# Calculo las Unidades Vendidas y Precio de los Productos del Mes de Mayo
Analisis_de_productos1 = Analisis_de_productos.sort_values(by='Cantidad Pedida', ascending=False)
Analisis_de_productos1 = Analisis_de_productos1.reset_index(inplace=False)
Analisis_de_productos1
Productos = Analisis_de_productos1['Producto']
Unidades = Analisis_de_productos1['Cantidad Pedida']
Precio = Analisis_de_productos1['Precio Unitario']

# Calculo las Unidades Vendidas y por Hora del Mes de Mayo
df1_ventas = dataframes['Mayo']
df1_ventas['Fecha de Compra'] = pd.to_datetime(df1_ventas['Fecha de Pedido'])
df1_ventas
def extraer_datos(datos):
    datos['Hora'] = datos['Fecha de Compra'].dt.hour
    return datos
df2_ventas = extraer_datos(df1_ventas)
df2_ventas
horas = list(range(24))
ventas_por_hora_mesMayo = df2_ventas.groupby('Hora')['Cantidad Pedida'].sum()

# Calculo las Unidades Vendidas por Dia del Mes de Mayo
def extraer_datos(datos):
    datos['Dia'] = datos['Fecha de Compra'].dt.day
    return datos
df3_ventas = extraer_datos(df2_ventas)
df3_ventas
ventas_por_dia = df3_ventas.groupby('Dia')['Cantidad Pedida'].sum()
dias = list(ventas_por_dia.index)

# Calculo el Valor Total de los Productos Pedidos por Dia del Mes de Mayo
valor_total_por_dia_Mayo = df1_ventas.groupby('Dia')['Cantidad Pedida', 'Precio Unitario'].sum()
valor_total_por_dia_Mayo['Valor Total'] = valor_total_por_dia_Mayo['Cantidad Pedida'] * valor_total_por_dia_Mayo['Precio Unitario']
dias1 = list(valor_total_por_dia_Mayo.index)
df4_ventas = valor_total_por_dia_Mayo

# Gráfico de las Ventas por Estado del Mes de Mayo
fig = px.choropleth(ventas_por_estado_mesMayo, locations='Estado', locationmode='USA-states',
                    color='Cantidad Pedida', color_continuous_scale='RdYlGn', scope='usa',
                    title='Ventas por Estado en Mayo'
                    )
fig.show()

# Gráfico de las Unidades Vendidas y Precio de los Productos del Mes de Mayo
def grafico_unidades_precio(productos, unidades, precio):
    fig = go.Figure()
    fig.add_trace(go.Bar(
        x=productos,
        y=unidades,
        name='Unidades Vendidas',
        marker=dict(color=unidades)
    ))
    fig.add_trace(go.Scatter(
        x=productos,
        y=precio,
        name='Precio Unitario',
        line=dict(color='Red', shape='spline', smoothing=1.3, width=3),
        yaxis='y2'
    ))
    fig.update_layout(
        title='Unidades Vendidas y Precio de Productos Mes de Mayo',
        xaxis=dict(title='Productos'),
        yaxis=dict(title='Unidades', side='left'),
        yaxis2=dict(title='Precio', overlaying='y', side='right'),
        yaxis_range=[0, max(unidades) * 1.1],
        yaxis2_range=[0, max(precio) * 1.1 ]
    )
    fig.show()

# Gráfico de las Unidades Vendidas por Hora del Mes de Mayo
def grafico_unidades_por_hora(horas, ventas_por_hora_mesMayo):
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=horas,
        y=ventas_por_hora_mesMayo,
        name='Ventas por Hora',
        line=dict(color='Red', shape='spline', smoothing=0.1, width=2)
    ))
    fig.update_layout(
        title='Unidades Vendidas por Hora Mes de Mayo',
        xaxis=dict(title='Horas', tickvals=horas),
        yaxis=dict(title='Unidades Vendidas', side='left')
    )
    fig.show()

# Gráfico de las Unidades Vendidas por Día del Mes de Mayo
def grafico_unidades_por_dia(dias, ventas_por_dia):
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=dias,
        y=ventas_por_dia,
        name='Ventas por Día',
        line=dict(color='Red', shape='spline', smoothing=0.5, width=2)
    ))
    fig.update_layout(
        title='Unidades Vendidas por Día Mes de Mayo',
        xaxis=dict(title='Días', tickvals=dias),
        yaxis=dict(title='Unidades Vendidas', side='left')
    )
    fig.show()

# Gráfico del Valor Total de los Productos Pedidos por Día del Mes de Mayo
def grafico_valor_total(dias1, valor_total_por_dia_Mayo):
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=dias1,
        y=valor_total_por_dia_Mayo['Valor Total'],
        marker=dict(size=6),
        line=dict(color='Red', shape='spline', smoothing=0.5, width=2)
    ))
    fig.update_layout(
        title='Valor Total de los Productos Pedidos por Día del Mes de Mayo',
        xaxis=dict(title='Día', dtick=1),
        yaxis=dict(title='Valor Total'),
        showlegend=False
    )
    fig.show()

# Llamo a las funciones para mostrar los gráficos
grafico_unidades_precio(Productos, Unidades, Precio)
grafico_unidades_por_hora(horas, ventas_por_hora_mesMayo)
grafico_unidades_por_dia(dias, ventas_por_dia)
grafico_valor_total(dias1, valor_total_por_dia_Mayo)


<ipython-input-11-31ba71a8b20e>:45: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



In [37]:
df_valorTotal_Mayo = valor_total_por_dia_Mayo['Valor Total'].sum()
print("Valor Total del Mes de Mayo: $", df_valorTotal_Mayo)

Valor Total del Mes de Mayo: $ 1877149405


In [12]:
pd.set_option('display.float_format', '{:.2f}'.format)

Analisis_de_productos = dataframes['Junio'].groupby('Producto').agg({
    'Cantidad Pedida' : 'sum',
    'Precio Unitario' : 'mean'
})
# Calculo los Productos Pedidos por Estado en el Mes de Junio
direccion_df = dataframes['Junio']['Dirección de Envio'].str.split(', ', expand=True)
direccion_df.columns = ['Calle', 'Ciudad', 'Cod Post Estado']
direccion_df[['Estado', 'Cod Post']] = direccion_df['Cod Post Estado'].str.split(' ', expand=True)
df_ventas_desglosado = pd.concat([dataframes['Junio'], direccion_df], axis=True)
ventas_por_estado_mesJunio = df_ventas_desglosado.groupby('Estado')['Cantidad Pedida'].sum()
ventas_por_estado_mesJunio = ventas_por_estado_mesJunio.reset_index(inplace=False)

# Calculo las Unidades Vendidas y Precio de los Productos del Mes de Junio
Analisis_de_productos1 = Analisis_de_productos.sort_values(by='Cantidad Pedida', ascending=False)
Analisis_de_productos1 = Analisis_de_productos1.reset_index(inplace=False)
Analisis_de_productos1
Productos = Analisis_de_productos1['Producto']
Unidades = Analisis_de_productos1['Cantidad Pedida']
Precio = Analisis_de_productos1['Precio Unitario']

# Calculo las Unidades Vendidas y por Hora del Mes de Junio
df1_ventas = dataframes['Junio']
df1_ventas['Fecha de Compra'] = pd.to_datetime(df1_ventas['Fecha de Pedido'])
df1_ventas
def extraer_datos(datos):
    datos['Hora'] = datos['Fecha de Compra'].dt.hour
    return datos
df2_ventas = extraer_datos(df1_ventas)
df2_ventas
horas = list(range(24))
ventas_por_hora_mesJunio = df2_ventas.groupby('Hora')['Cantidad Pedida'].sum()

# Calculo las Unidades Vendidas por Dia del Mes de Junio
def extraer_datos(datos):
    datos['Dia'] = datos['Fecha de Compra'].dt.day
    return datos
df3_ventas = extraer_datos(df2_ventas)
df3_ventas
ventas_por_dia = df3_ventas.groupby('Dia')['Cantidad Pedida'].sum()
dias = list(ventas_por_dia.index)

# Calculo el Valor Total de los Productos Pedidos por Dia del Mes de Junio
valor_total_por_dia_Junio = df1_ventas.groupby('Dia')['Cantidad Pedida', 'Precio Unitario'].sum()
valor_total_por_dia_Junio['Valor Total'] = valor_total_por_dia_Junio['Cantidad Pedida'] * valor_total_por_dia_Junio['Precio Unitario']
dias1 = list(valor_total_por_dia_Junio.index)
df4_ventas = valor_total_por_dia_Junio

# Gráfico de las Ventas por Estado del Mes de Junio
fig = px.choropleth(ventas_por_estado_mesJunio, locations='Estado', locationmode='USA-states',
                    color='Cantidad Pedida', color_continuous_scale='RdYlGn', scope='usa',
                    title='Ventas por Estado en Junio'
                    )
fig.show()

# Gráfico de las Unidades Vendidas y Precio de los Productos del Mes de Junio
def grafico_unidades_precio(productos, unidades, precio):
    fig = go.Figure()
    fig.add_trace(go.Bar(
        x=productos,
        y=unidades,
        name='Unidades Vendidas',
        marker=dict(color=unidades)
    ))
    fig.add_trace(go.Scatter(
        x=productos,
        y=precio,
        name='Precio Unitario',
        line=dict(color='Red', shape='spline', smoothing=1.3, width=3),
        yaxis='y2'
    ))
    fig.update_layout(
        title='Unidades Vendidas y Precio de Productos Mes de Junio',
        xaxis=dict(title='Productos'),
        yaxis=dict(title='Unidades', side='left'),
        yaxis2=dict(title='Precio', overlaying='y', side='right'),
        yaxis_range=[0, max(unidades) * 1.1],
        yaxis2_range=[0, max(precio) * 1.1 ]
    )
    fig.show()

# Gráfico de las Unidades Vendidas por Hora del Mes de Junio
def grafico_unidades_por_hora(horas, ventas_por_hora_mesJunio):
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=horas,
        y=ventas_por_hora_mesJunio,
        name='Ventas por Hora',
        line=dict(color='Red', shape='spline', smoothing=0.1, width=2)
    ))
    fig.update_layout(
        title='Unidades Vendidas por Hora Mes de Junio',
        xaxis=dict(title='Horas', tickvals=horas),
        yaxis=dict(title='Unidades Vendidas', side='left')
    )
    fig.show()

# Gráfico de las Unidades Vendidas por Día del Mes de Junio
def grafico_unidades_por_dia(dias, ventas_por_dia):
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=dias,
        y=ventas_por_dia,
        name='Ventas por Día',
        line=dict(color='Red', shape='spline', smoothing=0.5, width=2)
    ))
    fig.update_layout(
        title='Unidades Vendidas por Día Mes de Junio',
        xaxis=dict(title='Días', tickvals=dias),
        yaxis=dict(title='Unidades Vendidas', side='left')
    )
    fig.show()

# Gráfico del Valor Total de los Productos Pedidos por Día del Mes de Junio
def grafico_valor_total(dias1, valor_total_por_dia_Junio):
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=dias1,
        y=valor_total_por_dia_Junio['Valor Total'],
        marker=dict(size=6),
        line=dict(color='Red', shape='spline', smoothing=0.5, width=2)
    ))
    fig.update_layout(
        title='Valor Total de los Productos Pedidos por Día del Mes de Junio',
        xaxis=dict(title='Día', dtick=1),
        yaxis=dict(title='Valor Total'),
        showlegend=False
    )
    fig.show()

# Llamo a las funciones para mostrar los gráficos
grafico_unidades_precio(Productos, Unidades, Precio)
grafico_unidades_por_hora(horas, ventas_por_hora_mesJunio)
grafico_unidades_por_dia(dias, ventas_por_dia)
grafico_valor_total(dias1, valor_total_por_dia_Junio)


<ipython-input-12-2a5d25a9a1e7>:45: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



In [35]:
df_valorTotal_Junio = valor_total_por_dia_Junio['Valor Total'].sum()
print("Valor Total del Mes de Junio: $", df_valorTotal_Junio)

Valor Total del Mes de Junio: $ 1300131915


In [13]:
pd.set_option('display.float_format', '{:.2f}'.format)

Analisis_de_productos = dataframes['Julio'].groupby('Producto').agg({
    'Cantidad Pedida' : 'sum',
    'Precio Unitario' : 'mean'
})
# Calculo los Productos Pedidos por Estado en el Mes de Julio
direccion_df = dataframes['Julio']['Dirección de Envio'].str.split(', ', expand=True)
direccion_df.columns = ['Calle', 'Ciudad', 'Cod Post Estado']
direccion_df[['Estado', 'Cod Post']] = direccion_df['Cod Post Estado'].str.split(' ', expand=True)
df_ventas_desglosado = pd.concat([dataframes['Julio'], direccion_df], axis=True)
ventas_por_estado_mesJulio = df_ventas_desglosado.groupby('Estado')['Cantidad Pedida'].sum()
ventas_por_estado_mesJulio = ventas_por_estado_mesJulio.reset_index(inplace=False)

# Calculo las Unidades Vendidas y Precio de los Productos del Mes de Julio
Analisis_de_productos1 = Analisis_de_productos.sort_values(by='Cantidad Pedida', ascending=False)
Analisis_de_productos1 = Analisis_de_productos1.reset_index(inplace=False)
Analisis_de_productos1
Productos = Analisis_de_productos1['Producto']
Unidades = Analisis_de_productos1['Cantidad Pedida']
Precio = Analisis_de_productos1['Precio Unitario']

# Calculo las Unidades Vendidas y por Hora del Mes de Julio
df1_ventas = dataframes['Julio']
df1_ventas['Fecha de Compra'] = pd.to_datetime(df1_ventas['Fecha de Pedido'])
df1_ventas
def extraer_datos(datos):
    datos['Hora'] = datos['Fecha de Compra'].dt.hour
    return datos
df2_ventas = extraer_datos(df1_ventas)
df2_ventas
horas = list(range(24))
ventas_por_hora_mesJulio = df2_ventas.groupby('Hora')['Cantidad Pedida'].sum()

# Calculo las Unidades Vendidas por Dia del Mes de Julio
def extraer_datos(datos):
    datos['Dia'] = datos['Fecha de Compra'].dt.day
    return datos
df3_ventas = extraer_datos(df2_ventas)
df3_ventas
ventas_por_dia = df3_ventas.groupby('Dia')['Cantidad Pedida'].sum()
dias = list(ventas_por_dia.index)

# Calculo el Valor Total de los Productos Pedidos por Dia del Mes de Julio
valor_total_por_dia_Julio = df1_ventas.groupby('Dia')['Cantidad Pedida', 'Precio Unitario'].sum()
valor_total_por_dia_Julio['Valor Total'] = valor_total_por_dia_Julio['Cantidad Pedida'] * valor_total_por_dia_Julio['Precio Unitario']
dias1 = list(valor_total_por_dia_Julio.index)
df4_ventas = valor_total_por_dia_Julio

# Gráfico de las Ventas por Estado del Mes de Julio
fig = px.choropleth(ventas_por_estado_mesJulio, locations='Estado', locationmode='USA-states',
                    color='Cantidad Pedida', color_continuous_scale='RdYlGn', scope='usa',
                    title='Ventas por Estado en Julio'
                    )
fig.show()

# Gráfico de las Unidades Vendidas y Precio de los Productos del Mes de Julio
def grafico_unidades_precio(productos, unidades, precio):
    fig = go.Figure()
    fig.add_trace(go.Bar(
        x=productos,
        y=unidades,
        name='Unidades Vendidas',
        marker=dict(color=unidades)
    ))
    fig.add_trace(go.Scatter(
        x=productos,
        y=precio,
        name='Precio Unitario',
        line=dict(color='Red', shape='spline', smoothing=1.3, width=3),
        yaxis='y2'
    ))
    fig.update_layout(
        title='Unidades Vendidas y Precio de Productos Mes de Julio',
        xaxis=dict(title='Productos'),
        yaxis=dict(title='Unidades', side='left'),
        yaxis2=dict(title='Precio', overlaying='y', side='right'),
        yaxis_range=[0, max(unidades) * 1.1],
        yaxis2_range=[0, max(precio) * 1.1 ]
    )
    fig.show()

# Gráfico de las Unidades Vendidas por Hora del Mes de Julio
def grafico_unidades_por_hora(horas, ventas_por_hora_mesJulio):
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=horas,
        y=ventas_por_hora_mesJulio,
        name='Ventas por Hora',
        line=dict(color='Red', shape='spline', smoothing=0.1, width=2)
    ))
    fig.update_layout(
        title='Unidades Vendidas por Hora Mes de Julio',
        xaxis=dict(title='Horas', tickvals=horas),
        yaxis=dict(title='Unidades Vendidas', side='left')
    )
    fig.show()

# Gráfico de las Unidades Vendidas por Día del Mes de Julio
def grafico_unidades_por_dia(dias, ventas_por_dia):
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=dias,
        y=ventas_por_dia,
        name='Ventas por Día',
        line=dict(color='Red', shape='spline', smoothing=0.5, width=2)
    ))
    fig.update_layout(
        title='Unidades Vendidas por Día Mes de Julio',
        xaxis=dict(title='Días', tickvals=dias),
        yaxis=dict(title='Unidades Vendidas', side='left')
    )
    fig.show()

# Gráfico del Valor Total de los Productos Pedidos por Día del Mes de Julio
def grafico_valor_total(dias1, valor_total_por_dia_Julio):
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=dias1,
        y=valor_total_por_dia_Julio['Valor Total'],
        marker=dict(size=6),
        line=dict(color='Red', shape='spline', smoothing=0.5, width=2)
    ))
    fig.update_layout(
        title='Valor Total de los Productos Pedidos por Día del Mes de Julio',
        xaxis=dict(title='Día', dtick=1),
        yaxis=dict(title='Valor Total'),
        showlegend=False
    )
    fig.show()

# Llamo a las funciones para mostrar los gráficos
grafico_unidades_precio(Productos, Unidades, Precio)
grafico_unidades_por_hora(horas, ventas_por_hora_mesJulio)
grafico_unidades_por_dia(dias, ventas_por_dia)
grafico_valor_total(dias1, valor_total_por_dia_Julio)


<ipython-input-13-b306faf3ad4d>:45: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



In [34]:
df_valorTotal_Julio = valor_total_por_dia_Julio['Valor Total'].sum()
print("Valor Total del Mes de Julio: $", df_valorTotal_Julio)

Valor Total del Mes de Julio: $ 1360077250


In [14]:
pd.set_option('display.float_format', '{:.2f}'.format)

Analisis_de_productos = dataframes['Agosto'].groupby('Producto').agg({
    'Cantidad Pedida' : 'sum',
    'Precio Unitario' : 'mean'
})
# Calculo los Productos Pedidos por Estado en el Mes de Agosto
direccion_df = dataframes['Agosto']['Dirección de Envio'].str.split(', ', expand=True)
direccion_df.columns = ['Calle', 'Ciudad', 'Cod Post Estado']
direccion_df[['Estado', 'Cod Post']] = direccion_df['Cod Post Estado'].str.split(' ', expand=True)
df_ventas_desglosado = pd.concat([dataframes['Agosto'], direccion_df], axis=True)
ventas_por_estado_mesAgosto = df_ventas_desglosado.groupby('Estado')['Cantidad Pedida'].sum()
ventas_por_estado_mesAgosto = ventas_por_estado_mesAgosto.reset_index(inplace=False)

# Calculo las Unidades Vendidas y Precio de los Productos del Mes de Agosto
Analisis_de_productos1 = Analisis_de_productos.sort_values(by='Cantidad Pedida', ascending=False)
Analisis_de_productos1 = Analisis_de_productos1.reset_index(inplace=False)
Productos = Analisis_de_productos1['Producto']
Unidades = Analisis_de_productos1['Cantidad Pedida']
Precio = Analisis_de_productos1['Precio Unitario']

# Calculo las Unidades Vendidas y por Hora del Mes de Agosto
df1_ventas = dataframes['Agosto']
df1_ventas['Fecha de Compra'] = pd.to_datetime(df1_ventas['Fecha de Pedido'])
def extraer_datos(datos):
    datos['Hora'] = datos['Fecha de Compra'].dt.hour
    return datos
df2_ventas = extraer_datos(df1_ventas)
horas = list(range(24))
ventas_por_hora_mesAgosto = df2_ventas.groupby('Hora')['Cantidad Pedida'].sum()

# Calculo las Unidades Vendidas por Día del Mes de Agosto
def extraer_datos(datos):
    datos['Dia'] = datos['Fecha de Compra'].dt.day
    return datos
df3_ventas = extraer_datos(df2_ventas)
ventas_por_dia = df3_ventas.groupby('Dia')['Cantidad Pedida'].sum()
dias = list(ventas_por_dia.index)

# Calculo el Valor Total de los Productos Pedidos por Día del Mes de Agosto
valor_total_por_dia_Agosto = df1_ventas.groupby('Dia')['Cantidad Pedida', 'Precio Unitario'].sum()
valor_total_por_dia_Agosto['Valor Total'] = valor_total_por_dia_Agosto['Cantidad Pedida'] * valor_total_por_dia_Agosto['Precio Unitario']
dias1 = list(valor_total_por_dia_Agosto.index)

# Gráfico de las Ventas por Estado del Mes de Agosto
fig = px.choropleth(ventas_por_estado_mesAgosto, locations='Estado', locationmode='USA-states',
                    color='Cantidad Pedida', color_continuous_scale='RdYlGn', scope='usa',
                    title='Ventas por Estado en Agosto'
                    )
fig.show()

# Gráfico de las Unidades Vendidas y Precio de los Productos del Mes de Agosto
def grafico_unidades_precio(productos, unidades, precio):
    fig = go.Figure()
    fig.add_trace(go.Bar(
        x=productos,
        y=unidades,
        name='Unidades Vendidas',
        marker=dict(color=unidades)
    ))
    fig.add_trace(go.Scatter(
        x=productos,
        y=precio,
        name='Precio Unitario',
        line=dict(color='Red', shape='spline', smoothing=1.3, width=3),
        yaxis='y2'
    ))
    fig.update_layout(
        title='Unidades Vendidas y Precio de Productos Mes de Agosto',
        xaxis=dict(title='Productos'),
        yaxis=dict(title='Unidades', side='left'),
        yaxis2=dict(title='Precio', overlaying='y', side='right'),
        yaxis_range=[0, max(unidades) * 1.1],
        yaxis2_range=[0, max(precio) * 1.1 ]
    )
    fig.show()

# Gráfico de las Unidades Vendidas por Hora del Mes de Agosto
def grafico_unidades_por_hora(horas, ventas_por_hora_mesAgosto):
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=horas,
        y=ventas_por_hora_mesAgosto,
        name='Ventas por Hora',
        line=dict(color='Red', shape='spline', smoothing=0.1, width=2)
    ))
    fig.update_layout(
        title='Unidades Vendidas por Hora Mes de Agosto',
        xaxis=dict(title='Horas', tickvals=horas),
        yaxis=dict(title='Unidades Vendidas', side='left')
    )
    fig.show()

# Gráfico de las Unidades Vendidas por Día del Mes de Agosto
def grafico_unidades_por_dia(dias, ventas_por_dia):
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=dias,
        y=ventas_por_dia,
        name='Ventas por Día',
        line=dict(color='Red', shape='spline', smoothing=0.5, width=2)
    ))
    fig.update_layout(
        title='Unidades Vendidas por Día Mes de Agosto',
        xaxis=dict(title='Días', tickvals=dias),
        yaxis=dict(title='Unidades Vendidas', side='left')
    )
    fig.show()

# Gráfico del Valor Total de los Productos Pedidos por Día del Mes de Agosto
def grafico_valor_total(dias1, valor_total_por_dia_Agosto):
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=dias1,
        y=valor_total_por_dia_Agosto['Valor Total'],
        marker=dict(size=6),
        line=dict(color='Red', shape='spline', smoothing=0.5, width=2)
    ))
    fig.update_layout(
        title='Valor Total de los Productos Pedidos por Día del Mes de Agosto',
        xaxis=dict(title='Día', dtick=1),
        yaxis=dict(title='Valor Total'),
        showlegend=False
    )
    fig.show()

# Llamo a las funciones para mostrar los gráficos
grafico_unidades_precio(Productos, Unidades, Precio)
grafico_unidades_por_hora(horas, ventas_por_hora_mesAgosto)
grafico_unidades_por_dia(dias, ventas_por_dia)
grafico_valor_total(dias1, valor_total_por_dia_Agosto)


<ipython-input-14-8e762121070a>:41: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



In [33]:
df_valorTotal_Agosto = valor_total_por_dia_Agosto['Valor Total'].sum()
print("Valor Total del Mes de Agosto: $", df_valorTotal_Agosto)

Valor Total del Mes de Agosto: $ 965594380


In [15]:
pd.set_option('display.float_format', '{:.2f}'.format)

Analisis_de_productos = dataframes['Septiembre'].groupby('Producto').agg({
    'Cantidad Pedida' : 'sum',
    'Precio Unitario' : 'mean'
})
# Calculo los Productos Pedidos por Estado en el Mes de Septiembre
direccion_df = dataframes['Septiembre']['Dirección de Envio'].str.split(', ', expand=True)
direccion_df.columns = ['Calle', 'Ciudad', 'Cod Post Estado']
direccion_df[['Estado', 'Cod Post']] = direccion_df['Cod Post Estado'].str.split(' ', expand=True)
df_ventas_desglosado = pd.concat([dataframes['Septiembre'], direccion_df], axis=True)
ventas_por_estado_mesSeptiembre = df_ventas_desglosado.groupby('Estado')['Cantidad Pedida'].sum()
ventas_por_estado_mesSeptiembre = ventas_por_estado_mesSeptiembre.reset_index(inplace=False)

# Calculo las Unidades Vendidas y Precio de los Productos del Mes de Septiembre
Analisis_de_productos1 = Analisis_de_productos.sort_values(by='Cantidad Pedida', ascending=False)
Analisis_de_productos1 = Analisis_de_productos1.reset_index(inplace=False)
Productos = Analisis_de_productos1['Producto']
Unidades = Analisis_de_productos1['Cantidad Pedida']
Precio = Analisis_de_productos1['Precio Unitario']

# Calculo las Unidades Vendidas y por Hora del Mes de Septiembre
df1_ventas = dataframes['Septiembre']
df1_ventas['Fecha de Compra'] = pd.to_datetime(df1_ventas['Fecha de Pedido'])
def extraer_datos(datos):
    datos['Hora'] = datos['Fecha de Compra'].dt.hour
    return datos
df2_ventas = extraer_datos(df1_ventas)
horas = list(range(24))
ventas_por_hora_mesSeptiembre = df2_ventas.groupby('Hora')['Cantidad Pedida'].sum()

# Calculo las Unidades Vendidas por Día del Mes de Septiembre
def extraer_datos(datos):
    datos['Dia'] = datos['Fecha de Compra'].dt.day
    return datos
df3_ventas = extraer_datos(df2_ventas)
ventas_por_dia = df3_ventas.groupby('Dia')['Cantidad Pedida'].sum()
dias = list(ventas_por_dia.index)

# Calculo el Valor Total de los Productos Pedidos por Día del Mes de Septiembre
valor_total_por_dia_Septiembre = df1_ventas.groupby('Dia')['Cantidad Pedida', 'Precio Unitario'].sum()
valor_total_por_dia_Septiembre['Valor Total'] = valor_total_por_dia_Septiembre['Cantidad Pedida'] * valor_total_por_dia_Septiembre['Precio Unitario']
dias1 = list(valor_total_por_dia_Septiembre.index)

# Gráfico de las Ventas por Estado del Mes de Septiembre
fig = px.choropleth(ventas_por_estado_mesSeptiembre, locations='Estado', locationmode='USA-states',
                    color='Cantidad Pedida', color_continuous_scale='RdYlGn', scope='usa',
                    title='Ventas por Estado en Septiembre'
                    )
fig.show()

# Gráfico de las Unidades Vendidas y Precio de los Productos del Mes de Septiembre
def grafico_unidades_precio(productos, unidades, precio):
    fig = go.Figure()
    fig.add_trace(go.Bar(
        x=productos,
        y=unidades,
        name='Unidades Vendidas',
        marker=dict(color=unidades)
    ))
    fig.add_trace(go.Scatter(
        x=productos,
        y=precio,
        name='Precio Unitario',
        line=dict(color='Red', shape='spline', smoothing=1.3, width=3),
        yaxis='y2'
    ))
    fig.update_layout(
        title='Unidades Vendidas y Precio de Productos Mes de Septiembre',
        xaxis=dict(title='Productos'),
        yaxis=dict(title='Unidades', side='left'),
        yaxis2=dict(title='Precio', overlaying='y', side='right'),
        yaxis_range=[0, max(unidades) * 1.1],
        yaxis2_range=[0, max(precio) * 1.1 ]
    )
    fig.show()

# Gráfico de las Unidades Vendidas por Hora del Mes de Septiembre
def grafico_unidades_por_hora(horas, ventas_por_hora_mesSeptiembre):
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=horas,
        y=ventas_por_hora_mesSeptiembre,
        name='Ventas por Hora',
        line=dict(color='Red', shape='spline', smoothing=0.1, width=2)
    ))
    fig.update_layout(
        title='Unidades Vendidas por Hora Mes de Septiembre',
        xaxis=dict(title='Horas', tickvals=horas),
        yaxis=dict(title='Unidades Vendidas', side='left')
    )
    fig.show()

# Gráfico de las Unidades Vendidas por Día del Mes de Septiembre
def grafico_unidades_por_dia(dias, ventas_por_dia):
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=dias,
        y=ventas_por_dia,
        name='Ventas por Día',
        line=dict(color='Red', shape='spline', smoothing=0.5, width=2)
    ))
    fig.update_layout(
        title='Unidades Vendidas por Día Mes de Septiembre',
        xaxis=dict(title='Días', tickvals=dias),
        yaxis=dict(title='Unidades Vendidas', side='left')
    )
    fig.show()

# Gráfico del Valor Total de los Productos Pedidos por Día del Mes de Septiembre
def grafico_valor_total(dias1, valor_total_por_dia_Septiembre):
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=dias1,
        y=valor_total_por_dia_Septiembre['Valor Total'],
        marker=dict(size=6),
        line=dict(color='Red', shape='spline', smoothing=0.5, width=2)
    ))
    fig.update_layout(
        title='Valor Total de los Productos Pedidos por Día del Mes de Septiembre',
        xaxis=dict(title='Día', dtick=1),
        yaxis=dict(title='Valor Total'),
        showlegend=False
    )
    fig.show()

# Llamo a las funciones para mostrar los gráficos
grafico_unidades_precio(Productos, Unidades, Precio)
grafico_unidades_por_hora(horas, ventas_por_hora_mesSeptiembre)
grafico_unidades_por_dia(dias, ventas_por_dia)
grafico_valor_total(dias1, valor_total_por_dia_Septiembre)


<ipython-input-15-c7d53ac73c80>:41: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



In [32]:
df_valorTotal_Septiembre = valor_total_por_dia_Septiembre['Valor Total'].sum()
print("Valor Total del Mes de Septiembre: $", df_valorTotal_Septiembre)

Valor Total del Mes de Septiembre: $ 909054579


In [16]:
pd.set_option('display.float_format', '{:.2f}'.format)

Analisis_de_productos = dataframes['Octubre'].groupby('Producto').agg({
    'Cantidad Pedida' : 'sum',
    'Precio Unitario' : 'mean'
})
# Calculo los Productos Pedidos por Estado en el Mes de Octubre
direccion_df = dataframes['Octubre']['Dirección de Envio'].str.split(', ', expand=True)
direccion_df.columns = ['Calle', 'Ciudad', 'Cod Post Estado']
direccion_df[['Estado', 'Cod Post']] = direccion_df['Cod Post Estado'].str.split(' ', expand=True)
df_ventas_desglosado = pd.concat([dataframes['Octubre'], direccion_df], axis=True)
ventas_por_estado_mesOctubre = df_ventas_desglosado.groupby('Estado')['Cantidad Pedida'].sum()
ventas_por_estado_mesOctubre = ventas_por_estado_mesOctubre.reset_index(inplace=False)

# Calculo las Unidades Vendidas y Precio de los Productos del Mes de Octubre
Analisis_de_productos1 = Analisis_de_productos.sort_values(by='Cantidad Pedida', ascending=False)
Analisis_de_productos1 = Analisis_de_productos1.reset_index(inplace=False)
Productos = Analisis_de_productos1['Producto']
Unidades = Analisis_de_productos1['Cantidad Pedida']
Precio = Analisis_de_productos1['Precio Unitario']

# Calculo las Unidades Vendidas y por Hora del Mes de Octubre
df1_ventas = dataframes['Octubre']
df1_ventas['Fecha de Compra'] = pd.to_datetime(df1_ventas['Fecha de Pedido'])
def extraer_datos(datos):
    datos['Hora'] = datos['Fecha de Compra'].dt.hour
    return datos
df2_ventas = extraer_datos(df1_ventas)
horas = list(range(24))
ventas_por_hora_mesOctubre = df2_ventas.groupby('Hora')['Cantidad Pedida'].sum()

# Calculo las Unidades Vendidas por Día del Mes de Octubre
def extraer_datos(datos):
    datos['Dia'] = datos['Fecha de Compra'].dt.day
    return datos
df3_ventas = extraer_datos(df2_ventas)
ventas_por_dia = df3_ventas.groupby('Dia')['Cantidad Pedida'].sum()
dias = list(ventas_por_dia.index)

# Calculo el Valor Total de los Productos Pedidos por Día del Mes de Octubre
valor_total_por_dia_Octubre = df1_ventas.groupby('Dia')['Cantidad Pedida', 'Precio Unitario'].sum()
valor_total_por_dia_Octubre['Valor Total'] = valor_total_por_dia_Octubre['Cantidad Pedida'] * valor_total_por_dia_Octubre['Precio Unitario']
dias1 = list(valor_total_por_dia_Octubre.index)

# Gráfico de las Ventas por Estado del Mes de Octubre
fig = px.choropleth(ventas_por_estado_mesOctubre, locations='Estado', locationmode='USA-states',
                    color='Cantidad Pedida', color_continuous_scale='RdYlGn', scope='usa',
                    title='Ventas por Estado en Octubre'
                    )
fig.show()

# Gráfico de las Unidades Vendidas y Precio de los Productos del Mes de Octubre
def grafico_unidades_precio(productos, unidades, precio):
    fig = go.Figure()
    fig.add_trace(go.Bar(
        x=productos,
        y=unidades,
        name='Unidades Vendidas',
        marker=dict(color=unidades)
    ))
    fig.add_trace(go.Scatter(
        x=productos,
        y=precio,
        name='Precio Unitario',
        line=dict(color='Red', shape='spline', smoothing=1.3, width=3),
        yaxis='y2'
    ))
    fig.update_layout(
        title='Unidades Vendidas y Precio de Productos Mes de Octubre',
        xaxis=dict(title='Productos'),
        yaxis=dict(title='Unidades', side='left'),
        yaxis2=dict(title='Precio', overlaying='y', side='right'),
        yaxis_range=[0, max(unidades) * 1.1],
        yaxis2_range=[0, max(precio) * 1.1 ]
    )
    fig.show()

# Gráfico de las Unidades Vendidas por Hora del Mes de Octubre
def grafico_unidades_por_hora(horas, ventas_por_hora_mesOctubre):
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=horas,
        y=ventas_por_hora_mesOctubre,
        name='Ventas por Hora',
        line=dict(color='Red', shape='spline', smoothing=0.1, width=2)
    ))
    fig.update_layout(
        title='Unidades Vendidas por Hora Mes de Octubre',
        xaxis=dict(title='Horas', tickvals=horas),
        yaxis=dict(title='Unidades Vendidas', side='left')
    )
    fig.show()

# Gráfico de las Unidades Vendidas por Día del Mes de Octubre
def grafico_unidades_por_dia(dias, ventas_por_dia):
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=dias,
        y=ventas_por_dia,
        name='Ventas por Día',
        line=dict(color='Red', shape='spline', smoothing=0.5, width=2)
    ))
    fig.update_layout(
        title='Unidades Vendidas por Día Mes de Octubre',
        xaxis=dict(title='Días', tickvals=dias),
        yaxis=dict(title='Unidades Vendidas', side='left')
    )
    fig.show()

# Gráfico del Valor Total de los Productos Pedidos por Día del Mes de Octubre
def grafico_valor_total(dias1, valor_total_por_dia_Octubre):
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=dias1,
        y=valor_total_por_dia_Octubre['Valor Total'],
        marker=dict(size=6),
        line=dict(color='Red', shape='spline', smoothing=0.5, width=2)
    ))
    fig.update_layout(
        title='Valor Total de los Productos Pedidos por Día del Mes de Octubre',
        xaxis=dict(title='Día', dtick=1),
        yaxis=dict(title='Valor Total'),
        showlegend=False
    )
    fig.show()

# Llamo a las funciones para mostrar los gráficos
grafico_unidades_precio(Productos, Unidades, Precio)
grafico_unidades_por_hora(horas, ventas_por_hora_mesOctubre)
grafico_unidades_por_dia(dias, ventas_por_dia)
grafico_valor_total(dias1, valor_total_por_dia_Octubre)


<ipython-input-16-29dac17b592a>:41: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



In [29]:
df_valorTotal_Octubre = valor_total_por_dia_Octubre['Valor Total'].sum()
print("Valor Total del Mes de Octubre: $", df_valorTotal_Octubre)

Valor Total del Mes de Octubre: $ 2713068569


In [17]:
pd.set_option('display.float_format', '{:.2f}'.format)

Analisis_de_productos = dataframes['Noviembre'].groupby('Producto').agg({
    'Cantidad Pedida' : 'sum',
    'Precio Unitario' : 'mean'
})
# Calculo los Productos Pedidos por Estado en el Mes de Noviembre
direccion_df = dataframes['Noviembre']['Dirección de Envio'].str.split(', ', expand=True)
direccion_df.columns = ['Calle', 'Ciudad', 'Cod Post Estado']
direccion_df[['Estado', 'Cod Post']] = direccion_df['Cod Post Estado'].str.split(' ', expand=True)
df_ventas_desglosado = pd.concat([dataframes['Noviembre'], direccion_df], axis=True)
ventas_por_estado_mesNoviembre = df_ventas_desglosado.groupby('Estado')['Cantidad Pedida'].sum()
ventas_por_estado_mesNoviembre = ventas_por_estado_mesNoviembre.reset_index(inplace=False)

# Calculo las Unidades Vendidas y Precio de los Productos del Mes de Noviembre
Analisis_de_productos1 = Analisis_de_productos.sort_values(by='Cantidad Pedida', ascending=False)
Analisis_de_productos1 = Analisis_de_productos1.reset_index(inplace=False)
Productos = Analisis_de_productos1['Producto']
Unidades = Analisis_de_productos1['Cantidad Pedida']
Precio = Analisis_de_productos1['Precio Unitario']

# Calculo las Unidades Vendidas y por Hora del Mes de Noviembre
df1_ventas = dataframes['Noviembre']
df1_ventas['Fecha de Compra'] = pd.to_datetime(df1_ventas['Fecha de Pedido'])
def extraer_datos(datos):
    datos['Hora'] = datos['Fecha de Compra'].dt.hour
    return datos
df2_ventas = extraer_datos(df1_ventas)
horas = list(range(24))
ventas_por_hora_mesNoviembre = df2_ventas.groupby('Hora')['Cantidad Pedida'].sum()

# Calculo las Unidades Vendidas por Día del Mes de Noviembre
def extraer_datos(datos):
    datos['Dia'] = datos['Fecha de Compra'].dt.day
    return datos
df3_ventas = extraer_datos(df2_ventas)
ventas_por_dia = df3_ventas.groupby('Dia')['Cantidad Pedida'].sum()
dias = list(ventas_por_dia.index)

# Calculo el Valor Total de los Productos Pedidos por Día del Mes de Noviembre
valor_total_por_dia_Noviembre = df1_ventas.groupby('Dia')['Cantidad Pedida', 'Precio Unitario'].sum()
valor_total_por_dia_Noviembre['Valor Total'] = valor_total_por_dia_Noviembre['Cantidad Pedida'] * valor_total_por_dia_Noviembre['Precio Unitario']
dias1 = list(valor_total_por_dia_Noviembre.index)

# Gráfico de las Ventas por Estado del Mes de Noviembre
fig = px.choropleth(ventas_por_estado_mesNoviembre, locations='Estado', locationmode='USA-states',
                    color='Cantidad Pedida', color_continuous_scale='RdYlGn', scope='usa',
                    title='Ventas por Estado en Noviembre'
                    )
fig.show()

# Gráfico de las Unidades Vendidas y Precio de los Productos del Mes de Noviembre
def grafico_unidades_precio(productos, unidades, precio):
    fig = go.Figure()
    fig.add_trace(go.Bar(
        x=productos,
        y=unidades,
        name='Unidades Vendidas',
        marker=dict(color=unidades)
    ))
    fig.add_trace(go.Scatter(
        x=productos,
        y=precio,
        name='Precio Unitario',
        line=dict(color='Red', shape='spline', smoothing=1.3, width=3),
        yaxis='y2'
    ))
    fig.update_layout(
        title='Unidades Vendidas y Precio de Productos Mes de Noviembre',
        xaxis=dict(title='Productos'),
        yaxis=dict(title='Unidades', side='left'),
        yaxis2=dict(title='Precio', overlaying='y', side='right'),
        yaxis_range=[0, max(unidades) * 1.1],
        yaxis2_range=[0, max(precio) * 1.1 ]
    )
    fig.show()

# Gráfico de las Unidades Vendidas por Hora del Mes de Noviembre
def grafico_unidades_por_hora(horas, ventas_por_hora_mesNoviembre):
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=horas,
        y=ventas_por_hora_mesNoviembre,
        name='Ventas por Hora',
        line=dict(color='Red', shape='spline', smoothing=0.1, width=2)
    ))
    fig.update_layout(
        title='Unidades Vendidas por Hora Mes de Noviembre',
        xaxis=dict(title='Horas', tickvals=horas),
        yaxis=dict(title='Unidades Vendidas', side='left')
    )
    fig.show()

# Gráfico de las Unidades Vendidas por Día del Mes de Noviembre
def grafico_unidades_por_dia(dias, ventas_por_dia):
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=dias,
        y=ventas_por_dia,
        name='Ventas por Día',
        line=dict(color='Red', shape='spline', smoothing=0.5, width=2)
    ))
    fig.update_layout(
        title='Unidades Vendidas por Día Mes de Noviembre',
        xaxis=dict(title='Días', tickvals=dias),
        yaxis=dict(title='Unidades Vendidas', side='left')
    )
    fig.show()

# Gráfico del Valor Total de los Productos Pedidos por Día del Mes de Noviembre
def grafico_valor_total(dias1, valor_total_por_dia_Noviembre):
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=dias1,
        y=valor_total_por_dia_Noviembre['Valor Total'],
        marker=dict(size=6),
        line=dict(color='Red', shape='spline', smoothing=0.5, width=2)
    ))
    fig.update_layout(
        title='Valor Total de los Productos Pedidos por Día del Mes de Noviembre',
        xaxis=dict(title='Día', dtick=1),
        yaxis=dict(title='Valor Total'),
        showlegend=False
    )
    fig.show()

# Llamo a las funciones para mostrar los gráficos
grafico_unidades_precio(Productos, Unidades, Precio)
grafico_unidades_por_hora(horas, ventas_por_hora_mesNoviembre)
grafico_unidades_por_dia(dias, ventas_por_dia)
grafico_valor_total(dias1, valor_total_por_dia_Noviembre)


<ipython-input-17-7b34d512f20a>:41: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



In [30]:
df_valorTotal_Noviembre = valor_total_por_dia_Noviembre['Valor Total'].sum()
print("Valor Total del Mes de Noviembre: $", df_valorTotal_Noviembre)

Valor Total del Mes de Noviembre: $ 2093768459


In [18]:
pd.set_option('display.float_format', '{:.2f}'.format)

Analisis_de_productos = dataframes['Diciembre'].groupby('Producto').agg({
    'Cantidad Pedida' : 'sum',
    'Precio Unitario' : 'mean'
})
# Calculo los Productos Pedidos por Estado en el Mes de Diciembre
direccion_df = dataframes['Diciembre']['Dirección de Envio'].str.split(', ', expand=True)
direccion_df.columns = ['Calle', 'Ciudad', 'Cod Post Estado']
direccion_df[['Estado', 'Cod Post']] = direccion_df['Cod Post Estado'].str.split(' ', expand=True)
df_ventas_desglosado = pd.concat([dataframes['Diciembre'], direccion_df], axis=True)
ventas_por_estado_mesDiciembre = df_ventas_desglosado.groupby('Estado')['Cantidad Pedida'].sum()
ventas_por_estado_mesDiciembre = ventas_por_estado_mesDiciembre.reset_index(inplace=False)

# Calculo las Unidades Vendidas y Precio de los Productos del Mes de Diciembre
Analisis_de_productos1 = Analisis_de_productos.sort_values(by='Cantidad Pedida', ascending=False)
Analisis_de_productos1 = Analisis_de_productos1.reset_index(inplace=False)
Productos = Analisis_de_productos1['Producto']
Unidades = Analisis_de_productos1['Cantidad Pedida']
Precio = Analisis_de_productos1['Precio Unitario']

# Calculo las Unidades Vendidas y por Hora del Mes de Diciembre
df1_ventas = dataframes['Diciembre']
df1_ventas['Fecha de Compra'] = pd.to_datetime(df1_ventas['Fecha de Pedido'])
def extraer_datos(datos):
    datos['Hora'] = datos['Fecha de Compra'].dt.hour
    return datos
df2_ventas = extraer_datos(df1_ventas)
horas = list(range(24))
ventas_por_hora_mesDiciembre = df2_ventas.groupby('Hora')['Cantidad Pedida'].sum()

# Calculo las Unidades Vendidas por Día del Mes de Diciembre
def extraer_datos(datos):
    datos['Dia'] = datos['Fecha de Compra'].dt.day
    return datos
df3_ventas = extraer_datos(df2_ventas)
ventas_por_dia = df3_ventas.groupby('Dia')['Cantidad Pedida'].sum()
dias = list(ventas_por_dia.index)

# Calculo el Valor Total de los Productos Pedidos por Día del Mes de Diciembre
valor_total_por_dia_Diciembre = df1_ventas.groupby('Dia')['Cantidad Pedida', 'Precio Unitario'].sum()
valor_total_por_dia_Diciembre['Valor Total'] = valor_total_por_dia_Diciembre['Cantidad Pedida'] * valor_total_por_dia_Diciembre['Precio Unitario']
dias1 = list(valor_total_por_dia_Diciembre.index)

# Gráfico de las Ventas por Estado del Mes de Diciembre
fig = px.choropleth(ventas_por_estado_mesDiciembre, locations='Estado', locationmode='USA-states',
                    color='Cantidad Pedida', color_continuous_scale='RdYlGn', scope='usa',
                    title='Ventas por Estado en Diciembre'
                    )
fig.show()

# Gráfico de las Unidades Vendidas y Precio de los Productos del Mes de Diciembre
def grafico_unidades_precio(productos, unidades, precio):
    fig = go.Figure()
    fig.add_trace(go.Bar(
        x=productos,
        y=unidades,
        name='Unidades Vendidas',
        marker=dict(color=unidades)
    ))
    fig.add_trace(go.Scatter(
        x=productos,
        y=precio,
        name='Precio Unitario',
        line=dict(color='Red', shape='spline', smoothing=1.3, width=3),
        yaxis='y2'
    ))
    fig.update_layout(
        title='Unidades Vendidas y Precio de Productos Mes de Diciembre',
        xaxis=dict(title='Productos'),
        yaxis=dict(title='Unidades', side='left'),
        yaxis2=dict(title='Precio', overlaying='y', side='right'),
        yaxis_range=[0, max(unidades) * 1.1],
        yaxis2_range=[0, max(precio) * 1.1 ]
    )
    fig.show()

# Gráfico de las Unidades Vendidas por Hora del Mes de Diciembre
def grafico_unidades_por_hora(horas, ventas_por_hora_mesDiciembre):
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=horas,
        y=ventas_por_hora_mesDiciembre,
        name='Ventas por Hora',
        line=dict(color='Red', shape='spline', smoothing=0.1, width=2)
    ))
    fig.update_layout(
        title='Unidades Vendidas por Hora Mes de Diciembre',
        xaxis=dict(title='Horas', tickvals=horas),
        yaxis=dict(title='Unidades Vendidas', side='left')
    )
    fig.show()

# Gráfico de las Unidades Vendidas por Día del Mes de Diciembre
def grafico_unidades_por_dia(dias, ventas_por_dia):
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=dias,
        y=ventas_por_dia,
        name='Ventas por Día',
        line=dict(color='Red', shape='spline', smoothing=0.5, width=2)
    ))
    fig.update_layout(
        title='Unidades Vendidas por Día Mes de Diciembre',
        xaxis=dict(title='Días', tickvals=dias),
        yaxis=dict(title='Unidades Vendidas', side='left')
    )
    fig.show()

# Gráfico del Valor Total de los Productos Pedidos por Día del Mes de Diciembre
def grafico_valor_total(dias1, valor_total_por_dia_Diciembre):
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=dias1,
        y=valor_total_por_dia_Diciembre['Valor Total'],
        marker=dict(size=6),
        line=dict(color='Red', shape='spline', smoothing=0.5, width=2)
    ))
    fig.update_layout(
        title='Valor Total de los Productos Pedidos por Día del Mes de Diciembre',
        xaxis=dict(title='Día', dtick=1),
        yaxis=dict(title='Valor Total'),
        showlegend=False
    )
    fig.show()

# Llamo a las funciones para mostrar los gráficos
grafico_unidades_precio(Productos, Unidades, Precio)
grafico_unidades_por_hora(horas, ventas_por_hora_mesDiciembre)
grafico_unidades_por_dia(dias, ventas_por_dia)
grafico_valor_total(dias1, valor_total_por_dia_Diciembre)


<ipython-input-18-25a30800e85f>:41: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



In [31]:
df_valorTotal_Diciembre = valor_total_por_dia_Diciembre['Valor Total'].sum()
print("Valor Total del Mes de Diciembre: $", df_valorTotal_Diciembre)

Valor Total del Mes de Diciembre: $ 4152949690
